In [1]:
import pynauty as pn
import cplex as cp

In [2]:
m = cp.Cplex("./cut_added_model_cplex.lp")
node = {}
col_node = {}
row_node = {}
edg_node = {}
edg_wght = {}
wght_color = {}
rhs_color = {}
node_color = {}
num_col = m.variables.get_num()
num_row = m.linear_constraints.get_num()
num_color = 0
num_node = 0
for i_col in range(num_col):
    node[i_col] = i_col
    col_node[i_col] = i_col
    node_color[i_col] = num_color
    num_node += 1
for i_row in range(num_row):
    rhs = m.linear_constraints.get_rhs(i_row)
    node[num_col + i_row] = num_col + i_row
    row_node[i_row] = num_col + i_row
    if rhs_color.get(rhs) is None:
        num_color += 1
        rhs_color[rhs] = num_color
    node_color[num_col + i_row] = rhs_color.get(rhs)
    num_node += 1
edg_cnt = 0
for i_col in range(num_col):
    index, value = m.variables.get_cols(i_col).unpack()
    for idx, val in zip(index, value):
        node[num_col + num_row + edg_cnt] = num_col + num_row + edg_cnt
        edg_node[edg_cnt] = num_col + num_row + edg_cnt
        edg_wght[edg_cnt] = val
        if wght_color.get(val) is None:
            num_color += 1
            wght_color[val] = num_color
        this_color = wght_color.get(val)
        node_color[num_col + num_row + edg_cnt] = this_color
        edg_cnt += 1
        num_node += 1

tri_part_graph = pn.Graph(num_node)
edg_cnt = 0
for i_col in range(num_col):
    index, value = m.variables.get_cols(i_col).unpack()
    connect_edg_node = []
    for idx, val in zip(index, value):
        connect_edg_node.append(num_col + num_row + edg_cnt)
        tri_part_graph.connect_vertex(num_col + num_row + edg_cnt, [num_col + idx])
        edg_cnt += 1
    tri_part_graph.connect_vertex(i_col, connect_edg_node)
color_sets = []
print(num_color)
for color in range(num_color):
    color_set = set()
    for node in range(num_node):
        n_color = node_color.get(node)
        if n_color == color:
            color_set.add(node)
    color_sets.append(color_set)
tri_part_graph.set_vertex_coloring(color_sets)
(generators, grpsize1, grpsize2, orbits, numorbits) = pn.autgrp(tri_part_graph)
print(orbits)

21
[0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 28, 28, 28, 28, 28, 28, 28, 28, 28, 37, 38, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 49, 50, 50, 50, 50, 50, 50, 50, 50, 50, 59, 59, 59, 59, 59, 59, 59, 59, 59, 68, 68, 68, 68, 68, 68, 68, 68, 68, 77, 78, 79, 79, 79, 79, 83, 84, 85, 85, 85, 85, 89, 89, 89, 89, 93, 94, 94, 94, 94, 98, 98, 98, 98, 102, 103, 104, 105, 106, 106, 106, 106, 110, 110, 110, 110, 114, 115, 116, 116, 116, 116, 120, 120, 120, 120, 124, 125, 125, 125, 125, 129, 129, 129, 129, 133, 134, 134, 134, 134, 138, 138, 138, 138, 142, 79, 78, 79, 79, 79, 83, 89, 89, 85, 89, 85, 85, 85, 84, 89, 98, 98, 94, 98, 94, 94, 94, 93, 98, 102, 103, 104, 110, 110, 106, 110, 106, 106, 106, 105, 110, 114, 120, 120, 116, 120, 116, 116, 116, 115, 120, 129, 129, 125, 129, 125, 125, 125, 124, 129, 138, 138, 134, 138, 134, 134, 134, 133, 138, 142, 79, 79, 78, 79, 79, 83, 89, 85, 89, 85, 89, 85, 85, 89, 84, 98, 94, 98, 94, 98, 94, 94, 98, 93, 102,